# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234678715578                   -0.50    8.0
  2   -7.250106129843       -1.81       -1.39    1.0
  3   -7.250988118468       -3.05       -1.82    4.0
  4   -7.250981088087   +   -5.15       -1.88    2.0
  5   -7.251316889515       -3.47       -2.47    2.0
  6   -7.251337930451       -4.68       -3.11    2.0
  7   -7.251338692865       -6.12       -3.52    3.0
  8   -7.251338790062       -7.01       -4.01    2.0
  9   -7.251338798417       -8.08       -4.76    3.0
 10   -7.251338798652       -9.63       -5.19    3.0
 11   -7.251338798699      -10.33       -5.65    2.0
 12   -7.251338798704      -11.31       -6.47    3.0
 13   -7.251338798705      -12.93       -6.90    2.0
 14   -7.251338798705      -14.05       -7.47    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06084962089732653
[ Info: Arnoldi iteration step 2: normres = 0.6090418691934447
[ Info: Arnoldi iteration step 3: normres = 0.7596627910394803
[ Info: Arnoldi iteration step 4: normres = 0.25274168053506646
[ Info: Arnoldi iteration step 5: normres = 0.47697179856186817
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.79e-02, 7.16e-02, 4.20e-01, 2.14e-01, 7.11e-03)
[ Info: Arnoldi iteration step 6: normres = 0.3161091981310802
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.64e-03, 1.08e-01, 2.50e-01, 1.16e-01, 5.11e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08973835255003489
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.06e-04, 1.09e-02, 1.92e-02, 5.23e-02, 6.34e-02)
[ Info: Arnoldi iteration step 8: normres = 0.0858075811429688
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (7.52e-